In [1]:
import numpy as np
from itertools import combinations

In [2]:
def make_deck():
    return [1,2,3,4,5,6,7,8,9,10,10,10,10]*4

In [3]:
def create_game():
    deck = make_deck()
    np.random.shuffle(deck)
    hand = deck[:7]
    val = list(filter((10).__ne__, deck[7:]))[:3]
    return hand, 100*val[0] + 10*val[1] + val[2]

In [8]:
ops = ['+', '-', '*', '/']
inverse_ops = {'+':'-', '-':'+', '*':'/', '/':'x'}

def solver(hand, val):
    if len(hand) == 0 and val == 0:
        return True, []
    hand_c = hand.copy()
    for card in hand_c:
        hand.remove(card)
        for op in ops:
            if op == '/':
                w, l = solver(hand, val / card)
            elif op == '-':
                w, l = solver(hand, val - card)
            elif op == '+':
                w, l = solver(hand, val + card)
            else:
                w, l = solver(hand, val * card)
            if w:
                l.append([card, op])
                return True, l
        hand.append(card)
    return False, []

In [22]:
def pretty_print(sol):
    pretty = str(sol[0][0]) if sol[0][1] == '-' else str(-sol[0][0])
    for i, (card, op) in enumerate(sol[1:]):
        if i == 0:
            pretty = pretty + ' ' + inverse_ops[op] + ' ' + str(card)
        else:    
            pretty = '(' + pretty + ') ' + inverse_ops[op] + ' ' + str(card)
    return pretty

In [36]:
def play(verbose = True):
    h, v = create_game()
    if verbose:
        print(str(v) + ': ' + str(h))
    status, sol = solver(h, v)
    if verbose:
        if status:
            print('Solution: ' + pretty_print(sol))
        else:
            print('No Solution Found')
        return
    else:
        return status, sol, h, v

In [37]:
N = 100
count = 0
for _ in range(N):
    status, sol, h, v = play(False)
    if status:
        count += 1
    else:
        print(str(v) + ': ' + str(h))
print(count/N)

795: [10, 9, 6, 3, 10, 10, 10]
293: [3, 5, 2, 1, 1, 1, 2]
345: [10, 10, 10, 10, 8, 7, 10]
744: [10, 10, 9, 10, 10, 7, 10]
0.96
